## 17. Letter Combinations of a Phone Number
- Description:
  <blockquote>
    Given a string containing digits from `2-9` inclusive, return all possible letter combinations that the number could represent. Return the answer in **any order**.
     
    A mapping of digits to letters (just like on the telephone buttons) is given below. Note that 1 does not map to any letters.
     
    ![Image](https://assets.leetcode.com/uploads/2022/03/15/1200px-telephone-keypad2svg.png)
     
    **Example 1:**
    **Input:** digits = "23"
    **Output:** ["ad","ae","af","bd","be","bf","cd","ce","cf"]
     
    **Example 2:**
    **Input:** digits = "2"
    **Output:** ["a","b","c"]
     
    **Constraints:**
     
    - `1 <= digits.length <= 4`
    - `digits[i]` is a digit in the range `['2', '9']`.
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/letter-combinations-of-a-phone-number/description/)

- Topics: Backtracking

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, Backtracking

Time Complexity:  O(3^N × 4^M × L)  ~ O(3^N × 4^M), since L is typically small
N = number of digits that map to 3 letters (digits 2, 3, 4, 5, 6, 8)
M = number of digits that map to 4 letters (digits 7, 9)
Each complete path requires O(L) work to build the string, where L is the length of digits

Space Complexity: O(3^N × 4^M)
Output storage: O(3^N × 4^M)

We store all possible combinations
Each combination is a string of length L
Total: O(3^N × 4^M × L)


Recursion stack: O(L)

Maximum depth equals length of digits
Each recursive call stores parameters
This is O(L) where L = len(digits)


Phone mapping: O(1)

Fixed size dictionary (only digits 2-9)




    Time complexity: O(4N⋅N), where N is the length of digits. Note that 4 in this expression is referring to the maximum value length in the hash map, and not to the length of the input.

    The worst-case is where the input consists of only 7s and 9s. In that case, we have to explore 4 additional paths for every extra digit. Then, for each combination, it costs up to N to build the combination. This problem can be generalized to a scenario where numbers correspond with up to M digits, in which case the time complexity would be O(MN⋅N). For the problem constraints, we're given, M=4, because of digits 7 and 9 having 4 letters each.

    Space complexity: O(N), where N is the length of digits.

    Not counting space used for the output, the extra space we use relative to input size is the space occupied by the recursion call stack. It will only go as deep as the number of digits in the input since whenever we reach that depth, we backtrack.

    As the hash map does not grow as the inputs grows, it occupies O(1) space.




- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
from typing import List


class Solution:
    def letterCombinations(self, digits: str) -> List[str]:
        # If the input is empty, immediately return an empty answer array
        if len(digits) == 0:
            return []

        result = []
        
        # Map all the digits to their corresponding letters
        letters = {
            "2": "abc",
            "3": "def",
            "4": "ghi",
            "5": "jkl",
            "6": "mno",
            "7": "pqrs",
            "8": "tuv",
            "9": "wxyz",
        }

        def backtrack(index, path):
            # If the path is the same length as digits, we have a complete combination
            if len(path) == len(digits):
                result.append("".join(path))
                return  # Backtrack

            # Get the letters that the current digit maps to, and loop through them
            for letter in letters[digits[index]]:
                # Make a choice - Add the letter to our current path
                path.append(letter)
                
                # Explore further - Move on to the next digit
                backtrack(index + 1, path)
                
                # Undo the choice - Backtrack by removing the letter before moving onto the next
                path.pop()

        # Initiate backtracking with an empty path and starting index of 0
        backtrack(0, [])
        
        return result

### Solution 1.1, Trie, Additional constrain, the output should only contain strings in a dictionary that contains all the valid words.
Integrating a dictionary constraint into this problem is a classic way to turn a "brute force" backtracking problem into an efficient Pruning problem.

If you have a set of valid words, generating all possible combinations first and then filtering them would be highly inefficient (especially for long digit strings). Instead, you should prune the search tree early: if a prefix cannot possibly form a valid word, stop exploring that branch.

The Strategy: Pruning with a Trie

A Trie is the ideal data structure here because it allows you to check in O(1) or O(L) time whether a specific sequence of letters is a valid prefix of any word in your dictionary.
Key Modifications:

- Preprocessing: Load your dictionary into a Trie.

- Validation: In each step of your backtrack function, check if the current path exists as a prefix in the Trie.

- Termination: If the current path isn't a prefix, return immediately (prune). If you reach the end of the digits, only add the string to result if it is marked as a "complete word" in the Trie.****

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {} # char : TrieNode()
        self.is_word_end = False

class Solution:
    def letterCombinations(self, digits: str, dictionary: List[str]) -> List[str]:
        if not digits:
            return []

        # 1. Build the Trie from the dictionary
        root = TrieNode()
        for word in dictionary:
            node = root
            for char in word:
                if char not in node.children:
                    node.children[char] = TrieNode()
                node = node.children[char]
            node.is_word_end = True

        result = []
        num_letters = {
            "2":"abc", "3":"def", "4":"ghi", "5":"jkl",
            "6":"mno", "7":"pqrs", "8":"tuv", "9":"wxyz",
        }

        def backtrack(index, path, current_node):
            # If the current path is not a prefix in our Trie, stop searching
            if not current_node:
                return

            # Base case: we've used all digits
            if index == len(digits):
                if current_node.is_word:
                    result.append("".join(path))
                return
            
            # Explore possible letters for the current digit
            for letter in num_letters[digits[index]]:
                if letter in current_node.children:
                    path.append(letter)
                    # Pass the next Trie node to avoid re-traversing from the root
                    backtrack(index + 1, path, current_node.children[letter])
                    path.pop()

        backtrack(0, [], root)
        return result

### Solution 2, Iterative BFS
Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
from collections import deque


class Solution:
    def letterCombinations(self, digits: str) -> List[str]:
        output = deque()
        
        if not len(digits):
            return list(output)

        output.append("")

        char_map = ['0', '1', "abc", "def", "ghi",
                    "jkl", "mno", "pqrs", "tuv", "wxyz"]

        i = 0

        while(i < len(digits)):
            index = int(digits[i])

            while len(output[0]) == i:
                permutation = output.popleft()

                for ch in char_map[index]:
                    output.append(permutation+ch)

            i += 1

        return list(output)

""" 
    Complexity Analysis
    Time complexity: O(4^N)
    Space complexity: O(N)
    
    Total combinations = (choices for digit 1) × (choices for digit 2) × … × (choices for digit n)
    Since each digit gives at least 3 choices, and up to 4, the total grows exponentially with the number of digits.
    
    Each digit you process branches into multiple choices:

    Digit "2" → 'a', 'b', 'c' → 3 branches
    Digit "7" → 'p', 'q', 'r', 's' → 4 branches

    So, for each digit, you multiply the number of possible paths by the number of letters it maps to.

    Time complexity: O(4N⋅N), where N is the length of digits. Note that 4 in this expression is referring to the maximum value length in the hash map, and not to the length of the input.
    The worst-case is where the input consists of only 7s and 9s. In that case, we have to explore 4 additional paths for every extra digit. Then, for each combination, it costs up to N to build the combination. This problem can be generalized to a scenario where numbers correspond with up to M digits, in which case the time complexity would be O(MN⋅N). For the problem constraints, we're given, M=4, because of digits 7 and 9 having 4 letters each.

    Space complexity: O(N), where N is the length of digits.
    Not counting space used for the output, the extra space we use relative to input size is the space occupied by the recursion call stack. It will only go as deep as the number of digits in the input since whenever we reach that depth, we backtrack.
    As the hash map does not grow as the inputs grows, it occupies O(1) space.
    
    ⏱️ Time Complexity Analysis
        Key Idea:

            Time = (Number of combinations) × (Cost to build each combination)

            How many combinations are there?
                Each digit maps to 3 or 4 letters.
                    Digits "2"–"6", "8" → 3 letters
                    Digits "7", "9" → 4 letters
                If the input has n digits, and:
                    N = count of digits with 3 options
                    M = count of digits with 4 options
                    → Total combinations = 3ᴺ × 4ᴹ

            In worst case (all digits are "7" or "9"), this is O(4ⁿ).

            How much work per combination?
                At the leaf (when len(path) == len(digits)), we do:

                combinations.append("".join(path))

                Apply Code
                    "".join(path) takes O(n) time (to build a string of length n).
                But note: the total number of recursive calls is dominated by the number of nodes in the recursion tree.

            However, a tighter (and standard) way:
            Since every combination must be generated, and each has length n, the total output size is O(n × 3ᴺ × 4ᴹ).

            But in complexity analysis for such problems, we often express time in terms of the number of combinations, assuming string building is part of output cost.

            ✅ Standard accepted time complexity:

                O(3ᴺ × 4ᴹ) — or simply O(4ⁿ) in worst case (since 4 > 3).

            (Some sources include the n factor: O(n × 4ⁿ). Both are seen, but O(4ⁿ) is common when focusing on branching factor.)

        💾 Space Complexity Analysis

        We consider auxiliary space (excluding the output list).

            Recursion stack depth:
                We recurse once per digit → max depth = n
                → O(n)

            path list:
                Stores at most n characters → O(n)

            Output list (combinations):
                Contains all results → O(3ᴺ × 4ᴹ × n)
                But this is not counted in auxiliary space complexity (only extra space used during computation).

        ✅ So, auxiliary space complexity = O(n)
        (due to recursion stack + current path)

            📝 Note: If the problem asks for total space including output, then it’s O(n × 4ⁿ). But usually, we report extra space.

    """